In [16]:
import pandas as pd 
from bs4 import BeautifulSoup
import numpy as np 
import requests
from googlesearch import search 


In [28]:
wiki = BeautifulSoup(
    requests.get(
    'https://en.wikipedia.org/wiki/List_of_cities_in_India_by_population', 
    ).content
)

places = []
for location in wiki.find('tbody').find_all('tr'):
    try:
        places.append(location.find('a').text.lower()) 
    except Exception as e:
        pass 

places.pop(0)

'[3]'

In [29]:
data_dict = {
    'place': [], 
    'location':[], 
    'description':[], 
}

i = 0
for city in places:  
    URL = f'https://www.holidify.com/places/{city}/sightseeing-and-things-to-do.html'

    page = BeautifulSoup(
        requests.get(
            URL
        ).content, 'lxml'
    )

    for card in page.find_all('div', attrs = {'class':'card content-card'}):
        data_dict['place'].append(card.find('h3', attrs = {'class':'card-heading'}).text.split('.')[1].strip()) 
        data_dict['location'].append(f'{city}')
        data_dict['description'].append(card.find('p', attrs = {'class':'card-text'}).text)
        



In [31]:
places = pd.DataFrame(data_dict)

places

,place,location,description
0,Elephanta Caves,mumbai,"An UNESCO World Heritage Site, Elephanta Cave..."
1,"Marine Drive, Mumbai",mumbai,Starting from the north of Nariman Point and ...
2,Juhu Beach,mumbai,"The most popular beach in Mumbai, Juhu beach ..."
3,Colaba Causeway,mumbai,One of the most crowded and electrifying plac...
4,Gateway of India,mumbai,One of the most distinguished monument in Mum...
...,...,...,...
3210,Yak Safari,gangtok,Experiencing Yak Ride is one of the most esse...
3211,Mountain Biking in Gangtok,gangtok,Another one of the thrilling activities and t...
3212,9’ine Native Cuisine,gangtok,9’ine Native Cuisine is a dainty cafe located...
3213,Baker's Cafe,gangtok,Baker's Cafe is one of the most popular and p...


In [33]:
#Getting location data 

URL = 'https://www.google.com/search?q={}+latitude+and+longitude&client=opera&hs=ETW&sxsrf=AJOqlzVTXsc5PO1IscihNvz8yvfn07Bbkg%3A1677064148484&ei=1Pf1Y-GeHcKp4-EP4IKJ0A8&oq=Mumbai+latitude&gs_lcp=Cgxnd3Mtd2l6LXNlcnAQARgAMgoIABCABBBGEPsBMgUIABCABDIGCAAQFhAeMgkIABAWEB4Q8QQyCAgAEBYQHhAPMggIABAWEB4QDzIICAAQFhAeEA8yCQgAEBYQHhDxBDIGCAAQFhAeMgkIABAWEB4Q8QQ6CggAEEcQ1gQQsAM6BwgAELADEEM6DQgAEOQCENYEELADGAE6DwguENQCEMgDELADEEMYAjoMCC4QyAMQsAMQQxgCOgQIIxAnOgUIABCGAzoICAAQgAQQsQM6BAgAEAM6CAgAELEDEIMBOggILhCABBCxAzoQCC4QgAQQFBCHAhCxAxCDAToNCC4QgAQQFBCHAhCxAzoFCAAQkQI6BQguEJECOgsIABCABBCxAxCDAToQCAAQgAQQFBCHAhCxAxCDAUoECEEYAFCsAVi6JmDQMWgDcAB4AIABnAGIAeYOkgEEMC4xNJgBAKABAcgBE8ABAdoBBggBEAEYCdoBBggCEAEYCA&sclient=gws-wiz-serp'

page = BeautifulSoup(
    requests.get(
        URL
    ).content
)

page.find_all('div', attrs = {'class':'Z0LcW t2B5Cf'})

[]

In [39]:
URL = 'https://latitudelongitude.org/in/{}/'

co_ord = {
    'city':[], 
    'latitude':[], 
    'longitude':[]
}

count = 0
for city in places:
    SEARCH = URL.format(city)


    lat_long = BeautifulSoup(
        requests.get(
            SEARCH, 
            headers = {
                'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36 OPR/95.0.4635.25'
            }
        ).content, 
    )
    
    try:

        lat, long = lat_long.find(
            'section', 
            attrs = {
                'class':'intro'
            }
        ).find('span').text.split(',')

    except Exception as e:
        count += 1 
        
    co_ord['city'].append(city)
    co_ord['latitude'].append(float(lat))
    co_ord['longitude'].append(float(long))
   



In [40]:
cord = pd.DataFrame(co_ord)

dataset = pd.merge(
    left = places, 
    right = cord, 
    left_on = 'location', 
    right_on = 'city', 
    how = 'inner'
).drop(
    columns = [
    'city'
])

In [41]:
dataset.to_csv('ds/places.csv', index = False )

In [42]:
dataset.loc[dataset.location == 'mumbai', 'place'].unique()

array([], dtype=object)

In [46]:
places.to_csv('ds/places.csv', index = False )